<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />


# _Embeddings_ multilenguaje<a id="top"></a>

<i><small>Autor: Alberto Díaz Álvarez<br>Última actualización: 2023-03-14</small></i></div>
                                                  

***

## Introducción

Una de las aplicaciones más prometedoras de los _Embeddings_ es el aprendizaje de transferencia multilingüe.

Supongamos que desea entrenar un modelo de NLP en varios idiomas, pero sólo se dispone de datos de entrenamiento en uno de ellos. Recopilar nuevos datos de entrenamiento para cada una de las lenguas destino puede resultar costoso, y traducir todos los textos que se quieren procesar más todavía. Sin embargo, con los _embeddings_ multilingües se puede intentar transferir un modelo de un idioma a otro de forma más eficaz.

## Objetivos

Entrenaremos un _embedding_ para que sea capaz de determinar

## Imports y configuración

A continuación importaremos las librerías que se usarán a lo largo del notebook.

In [2]:
import os
import urllib
import zipfile

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from wordfreq import top_n_list

Asimismo, configuramos algunos parámetros para adecuar la presentación gráfica.

In [3]:
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (20, 6),'figure.dpi': 64})

***

## Conjunto de datos

Vamos a partir directamente de dos _embeddings_ extraídos de la biblioteca [Muse](https://github.com/facebookresearch/MUSE) (perteneciente a [Facebook Research](https://research.fb.com/)).

Se trata de un respositorio que contiene _embeddings_ entrenados con la Wikipedia para más de 30 idiomas alineados en un único espacio vectorial. Nos vamos a centrar en los idiomas inglés y español.

In [5]:
WIKI_EN = 'tmp/wiki.en.vec'
WIKI_ES = 'tmp/wiki.es.vec'

if not os.path.exists('tmp/wiki.en.vec'):
    print('Downloading english...', end='')
    urllib.request.urlretrieve('https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec', WIKI_EN)
    print('OK')
if not os.path.exists('wiki.es.vec'):
    print('Downloading spanish...', end='')
    urllib.request.urlretrieve('https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.es.vec', WIKI_ES)
    print('OK')

Y cargamos los pesos y las palabras en sus respectivas variables

In [6]:
def load_embedding(path):
    print(f'Loading vectors from {path}')
    embedding = []
    word_index = {}
    with open(path, 'r', encoding='utf-8') as f:
        next(f)
        for i, line in enumerate(f):
            if i > 100000:
                break
            word, emb = line.rstrip().split(' ', 1)
            emb = np.fromstring(emb, sep=' ')
            embedding.append(emb)
            word_index[word] = len(word_index)

    embedding = np.vstack(embedding)
    return embedding, word_index

en_embedding, en_embedding_word_index = load_embedding(WIKI_EN)
es_embedding, es_embedding_word_index = load_embedding(WIKI_ES)

Loading vectors from tmp/wiki.en.vec
Loading vectors from tmp/wiki.es.vec


## Preprocesamiento de datos

Al igual que antes, entrenaremos un modelo sencillo de análisis de sentimiento con el conjunto de IMDb.

Tras descargar los datos, aplicaremos los pasos tradicionales de preprocesamiento. Trabajaremos con un vocabulario de 10.000 palabras, cortaremos los textos largos después de 256 palabras y rellenaremos todos los textos más cortos con padding.

In [7]:
VOCABULARY_SIZE = 10000
START_INDEX = 1
OOV_INDEX   = 2
INDEX_FROM  = 3
EMBEDDING_DIM = 300
SEQUENCE_LENGTH = 256

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(
    num_words=VOCABULARY_SIZE,  # Número de palabras que queremos en nuestro vocabulario,
    start_char=START_INDEX,     # Índice para el token de comienzo de secuencia
    oov_char=OOV_INDEX,         # Índice para el caracter desconocido (out of vocabulary)
    index_from=INDEX_FROM,      # Índice a partir del cual se empiezan a indexar las palabras
)

x_train = tf.keras.preprocessing.sequence.pad_sequences(
    x_train,
    padding='post',
    maxlen=SEQUENCE_LENGTH
)
x_test = tf.keras.preprocessing.sequence.pad_sequences(
    x_test,
    padding='post',
    maxlen=SEQUENCE_LENGTH
)

### _Embeddings_ preentrenados

Para poder transferir nuestro modelo entre idiomas, nuestra capa de _embedding_ necesita proyectar las palabras en un espacio vectorial multilingüe.

Por tanto, vamos a inicializar nuestra capa de _embedding_ con (un subconjunto de) las incrustaciones de palabras descargadas. En primer lugar, tenemos que averiguar qué palabras representan los índices de los datos IMDB preprocesados y después, tenemos que crear un _embedding_ donde cada fila contenga el vector de palabra indexad0 por su número de fila. 

In [8]:
def create_embedding_matrix(target_word_index, embedding_word_index, embedding, rows, cols):
    embedding_matrix = np.zeros((rows, cols))
    for word, index in target_word_index.items():
        if index < rows and word in embedding_word_index: 
            embedding_matrix[index] = embedding[embedding_word_index[word]]
    return embedding_matrix

en_word_index = tf.keras.datasets.imdb.get_word_index()
en_word_index = {word: (index + INDEX_FROM) for word, index in en_word_index.items()}
en_word_index['<PAD>']   = 0
en_word_index['<START>'] = START_INDEX
en_word_index['<UNK>']   = OOV_INDEX

en_embedding_matrix = create_embedding_matrix(
    target_word_index=en_word_index,
    embedding_word_index=en_embedding_word_index,
    embedding=en_embedding,
    rows=VOCABULARY_SIZE + INDEX_FROM-1,
    cols=EMBEDDING_DIM
)
en_embedding_matrix.shape

(10002, 300)

## Creando y entrenando nuestro modelo

Ahora es el momento de crear y entrenar nuestro modelo, que será prácticamente igual que el del ejemplo anterior pero habiendo cambiado el embedding

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(
        input_dim=VOCABULARY_SIZE + INDEX_FROM - 1,
        output_dim=EMBEDDING_DIM,
        input_length=SEQUENCE_LENGTH,
        weights=[en_embedding_matrix],
        trainable=False
    ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 256, 300)          3000600   
                                                                 
 flatten (Flatten)           (None, 76800)             0         
                                                                 
 dense (Dense)               (None, 8)                 614408    
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 3,615,017
Trainable params: 614,417
Non-trainable params: 3,000,600
_________________________________________________________________


Entrenamos el modelo:

In [10]:
history = model.fit(x_train, y_train, validation_split=0.1, epochs=100, verbose=0)

Y comprobamos qué tal ha evolucionado el entrenamiento

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title(f'Training: {history.history["loss"][-1]:.2f}, validation: {history.history["val_loss"][-1]:.2f}')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['binary_accuracy'], label='Training')
plt.plot(history.history['val_binary_accuracy'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title(f'Training: {history.history["binary_accuracy"][-1]:.2f}, validation: {history.history["val_binary_accuracy"][-1]:.2f}')
plt.legend()

plt.tight_layout()
plt.show()

Vamos a ver si el modelo hace lo que tiene que hacer

In [ ]:
comments = [
    'the movie was amazing'.split(),
    'i hated it'.split()
]
vectors = [
    [en_word_index.get(word, OOV_INDEX) for word in comment]
    for comment in comments
]
padded_vectors = tf.keras.preprocessing.sequence.pad_sequences(
    vectors,
    padding='post',
    maxlen=SEQUENCE_LENGTH
)

model.predict(padded_vectors)

## Transferencia de nuestro modelo

Obviamente, el modelo anterior no puede utilizarse para clasificar textos escritos en español. Para transferir nuestro modelo del inglés al español, tenemos que sustituir su embedding en inglés por una capa de embedding en español.

De nuevo vamos a trabajar con un vocabulario de 10.000 palabras. Desgraciadamente, no disponemos de un conjunto de textos en español relevantes en los que encajar el vocabulario, por lo que usaremos la librería [wordfreq](https://github.com/LuminosoInsight/wordfreq). Esta contiene información sobre la frecuencia de las palabras en muchas lenguas occidentales. En particular, `top_n_list` nos dará las $n$ palabras más frecuentes de un idioma.

In [ ]:
print(top_n_list('en', 10))
print(top_n_list('es', 10))

Construiremos nuestro vocabulario a partir de las 10.000 palabras españolas más frecuentes y lo utilizaremos para crear un embedding en español.

In [ ]:
es_word_index = {word: idx+INDEX_FROM for idx, word in enumerate(top_n_list('es', VOCABULARY_SIZE))}
en_word_index['<PAD>']   = 0
en_word_index['<START>'] = START_INDEX
en_word_index['<UNK>']   = OOV_INDEX

es_embedding_matrix = create_embedding_matrix(
    target_word_index=es_word_index,
    embedding_word_index=es_embedding_word_index,
    embedding=es_embedding,
    rows=VOCABULARY_SIZE + INDEX_FROM-1,
    cols=EMBEDDING_DIM
)
es_embedding_matrix.shape

Lo siguiente, vamos a reemplazar los pesos del embedding original (el inglés) por los pesos del nuevo embedding (el español). Para ello hay que usar el método `set_weights([weight_matrix])` del objeto de la clase `Embedding`.

In [ ]:
model.layers[0].set_weights([es_embedding_matrix])

Y como por arte de magie (magia rara) el modelo que acabamos de entrenar con textos en inglés ahora puede tomar textos en castellano como entrada y clasificarlos correctamente.

In [ ]:
comments = [
    'la película fue muy buena'.split(),
    'no me ha gustado nada'.split()
]
vectors        =  [
    [es_word_index.get(word, OOV_INDEX) for word in comment]
    for comment in comments
]
padded_vectors = tf.keras.preprocessing.sequence.pad_sequences(
    vectors,
    padding='post',
    maxlen=SEQUENCE_LENGTH
)
model.predict(padded_vectors)

## Conclusiones

Los _embeddings_ multilingües nos permiten transferir un modelo de un idioma a otro. Es muy útil cuando se necesita aplicar un mismo modelo a varios idiomas y únicamente se disponen de datos en uno de ellos.

Por supuesto, esto no está exento de problemas. Cuanto más diferentes son dos idiomas, peor se comportará (lo acabamos de ver con español e inglés). Los idiomas no sólo son palabras, sino también expresiones, órdenes de palabras, etc. Sin embargo, cuando dos idiomas son parecidos en términos lingüísticos (e.g. español y asturiano), esta solución es razonablemente buena.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>